# Far Field - Preprocess Raw Data

Open far-field raw data

In [1]:
import h5py
import numpy as np

h5_filename="/nfs/chess/raw/2023-3/id3a/berman-3804-a/zx30-t1-ff-0/1/ff/ff1_000231.h5"
f = h5py.File(h5_filename, 'r')
vol=f["imageseries"]["images"][:,:,:] # this is IO bounded
D,H,W=vol.shape
print("Loaded hdf5 raw data", vol.shape,vol.dtype)

Loaded hdf5 raw data (1445, 3888, 3072) uint16


Compute min and max projection

In [2]:
low =np.min(vol,axis=0)
high=np.max(vol,axis=0)

Show saturation

In [3]:
import tifffile
import numpy as np
import PIL

import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets


# ////////////////////////////////////////////////
def ShowSaturation(img):

    H,W=img.shape
    zmin,zmax=np.min(img),np.max(img)

    show_min        = widgets.BoundedIntText(value=0    , min=zmin, max=zmax, step=1, description=f'Min/{zmin}')
    show_max        = widgets.BoundedIntText(value=10000, min=zmin, max=zmax, step=1, description=f'Max/{zmax}')
    show_saturation = widgets.Checkbox(description='Show saturation: ',value=True)

    fig = go.Figure()
    fig.layout.dragmode='pan'

    fig.add_trace(go.Scatter(x=[0, W ],y=[0, H],mode="markers",marker_opacity=0)) # This trace is added to help the autoresize logic work.
    fig.update_xaxes(visible=False,range=[0, W])
    fig.update_yaxes(visible=False,range=[0, H],scaleanchor="x")

    # ////////////////////////////////////////////////
    def CreateImage(r=None,g=None,b=None):
        if r is None:
            gray=(np.random.random([256,256])*255.0).astype(np.uint8)
            return CreateImage(gray,gray,gray)
        return PIL.Image.fromarray(np.stack([r,g,b],axis=-1))


    fig.add_layout_image(dict(
        x=0, y=H, sizex=W, sizey=H,
        xref="x",yref="y",opacity=1.0, layer="below",sizing="stretch",
        source=CreateImage()
    ))
    fig.update_layout(margin={"l": 0, "r": 0, "t": 0, "b": 0})

    figure_widget=go.FigureWidget(fig)
    figure_widget.set_trait("_config",{"scrollZoom": True})  # if you cannot debug, comment this line

    def onChange(change=None):
        print(f"RefreshColors min={show_min.value} max={show_max.value} show_staturation={show_saturation.value} ")
        r=g=b=(255.0*(img/zmax)).astype(np.uint8)
        if show_saturation.value: 
            g,b=np.copy(r),np.copy(r)
            mask=r>(255.0*(show_max.value/zmax))
            r[mask],g[mask],b[mask]=255,0,0
        new_image=CreateImage(r,g,b)
        figure_widget.update_layout_images(source=new_image)


    show_min       .observe(onChange, names="value")
    show_max       .observe(onChange, names="value")
    show_saturation.observe(onChange, names="value")

    onChange()
    return widgets.HBox([figure_widget,widgets.VBox([show_min,show_max,show_saturation])])

ShowSaturation(high)

RefreshColors min=285 max=10000 show_staturation=True 


    'data': [{'marker': {'opacity': 0},
              'mode': 'markers',
       …

# Far Field - Reduced Data


In [ ]:
import os
import hexrd
from pp_dexela import PP_Dexela

# hardcoded
frame_start = 4
threshold  = 50
ostart     = 0. # omega start
oend       = 360.# omega end

data=f["imageseries"]["images"]
print(data)
D,H,W=data.shape

# get panel id (first 3 letters)
panel_id=os.path.basename(h5_filename)[0:3]
assert panel_id in ["ff1","ff2"]
print(D,H,W,panel_id)

panel_opts = {
    'ff1' : [('add-row', 1944), ('add-column', 1296),('flip', 'v'), ],
    'ff2' : [('add-row', 1944), ('add-column', 1296),('flip', 'h'), ]
}

num_frames = D-frame_start
omw = hexrd.imageseries.omega.OmegaWedges(num_frames)
omw.addwedge(ostart, oend, num_frames)
ppd = PP_Dexela(h5_filename, omw, panel_opts[panel_id], panel_id=panel_id, frame_start=frame_start)
processed = ppd.processed()
processed.metadata['omega']    = ppd.omegas
processed.metadata['panel_id'] = ppd.panel_id.upper()

# write all results to several NPZ file (sparse)
hexrd.imageseries.write(processed, "__dummy_filename", ppd.PROCFMT, style="npz", threshold=threshold, cache_file=f"test.npz")

Read the NPZ file (i.e. 3d Volume)

In [ ]:
reduced=np.load("test.npz")
# reduced=np.load("/nfs/chess/aux/reduced_data/cycles/2023-3/id3a/berman-3804-a/mg1y-1-ff-0/monolithic_cache_files/mg1y-1-ff-0_5_ff1_000030-cachefile.npz")
D=int(reduced['nframes'])
H,W=reduced['shape']
print(D,H,W)

assert(reduced['dtype'].tobytes().decode('ascii')=="float32")
assert(len(reduced['omega'])==D) # angle ranges from 0 to 360
assert(reduced['panel_id'] in ("FF1","FF2"))

# convert the sparse representation to a 3D volume
low,high=[np.zeros(shape=(H,W),dtype=np.float32) for I in range(2)]
for Z in range(D):
    for r,c,d in zip(reduced[f'{Z}_row'],reduced[f'{Z}_col'],reduced[f'{Z}_data']):
        low[r,c] =np.minimum(d,low [r,c]) if Z else d
        high[r,c]=np.maximum(d,high[r,c]) if Z else d

print(high.dtype,high.shape,np.min(high),np.max(high))

Show the preprocessed data

In [ ]:
ShowSaturation(high)